Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy_, valamint a _scipy.stats t_ és _f_ osztályai

In [2]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
from pathlib import Path

<IPython.core.display.Javascript object>

In [4]:
datalib = Path.cwd().parent.joinpath('data')

<IPython.core.display.Javascript object>

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.


3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot. <u>Amennyiben működött, önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályt is.</u>

7., Készíts egy új __LinearRegressionGLS__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami Feasible GLS elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között, amely az első változó legyen. <u>__(numpy.linalg.lstsq() nem használható)__</u>

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítsd ki az osztályt egy __get_wald_test_result__ metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Wald: wald_value, p-value: p_value__, ahol az wald_value és p_value helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. A függvény bemenete során feltételezzük, hogy r minden eleme 0, R-t listák listája formában adjuk át. A Wald statisztika p értékét az előző feladathoz hasonlóan számold ki, de figyelj, hogy a <u>teszt 1 oldalú</u>.

12., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

__Feasible GLS becslésének a menete__:
1.) A bementeni adatok alapján OLS elven becsüld meg a változók együtthatóit.
2.) Számold ki a hibatagokat
3.) A hibatagoknak vedd a négyzetét
4.) Becsülj egy új modellt, ahol a bal oldali változó a hibatag négyzetének logaritmusa, míg a jobboldali változók az eredeti jobb oldali változóid.
5.) Számold ki a becsült értékeket (logaritmikus négyzetes hibákat) és vedd őket az e kitevőjeként, majd vonj belőlük gyököt.
6.) A kapott vektornak vedd elemenként az inverzét és helyezd el egy diagonális mátrix főátlójában őket (np.diag). -> Az így kapott mátrix lesz a V inverz mátrix.
7.) A V inverz mátrix felhasználásával becsüld meg a GLS regresszió paramétereit
8.) Egyszerűsítés képpen számold ki a hibatagot a nem transzformált Y és X mátrixok és GLS paraméterek segítségével.